In [1]:
periodo = 'pandemia'

# Importe as bibliotecas necessárias

In [2]:
%matplotlib inline
from IPython.display import clear_output
import tqdm
import os
import pandas as pd
import numpy as np
import seaborn as sns
import random
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer 
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree._classes import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neighbors._classification import KNeighborsClassifier
from sklearn.neural_network._multilayer_perceptron import MLPClassifier
from sklearn.linear_model._logistic import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from yellowbrick.style.colors import resolve_colors
from sklearn.model_selection._validation import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from unidecode import unidecode
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.decomposition import PCA
import pickle
#import statsmodels.api as sm
import platform 
np.random.seed(42)

In [3]:
def strfdelta(tdelta, fmt="{hours}:{minutes}:{seconds}"):
    d = {"days": tdelta.days}
    d["hours"], rem = divmod(tdelta.seconds, 3600)
    d["minutes"], d["seconds"] = divmod(rem, 60)
    return fmt.format(**d)

# Carregando Base de dados

In [4]:
df = pd.read_csv(f'twitterbase_{periodo}.csv', sep=';')
df.head()

,volumeTweets_media,volumeTweets_variancia,volumeTweets_mediaMovelPonterada,volumeTweets_entropia,indiceInsonia_media,indiceInsonia_variancia,indiceInsonia_mediaMovelPonterada,indiceInsonia_entropia,pronome1Pessoa_media,pronome1Pessoa_variancia,...,links_entropia,midia_media,midia_variancia,midia_mediaMovelPonterada,midia_entropia,curtidas_media,curtidas_variancia,curtidas_mediaMovelPonterada,curtidas_entropia,classe
0,0.291005,0.587274,2.043956,0.651770,0.085979,0.080240,0.592033,0.309788,0.111111,0.268078,...,0.191444,0.111111,0.194004,0.741758,0.329227,0.174603,1.900731,1.137363,0.246161,depressao
1,4.523810,28.701814,31.454545,2.429842,0.942526,0.623469,6.755375,2.050928,1.333333,4.269841,...,0.729340,0.000000,0.000000,0.000000,0.000000,3.369048,28.423328,24.142857,2.111950,depressao
2,3.780269,176.637817,23.240741,1.897887,0.655502,1.126668,4.608926,1.508046,0.381166,1.330049,...,0.386422,0.152466,0.550745,0.884259,0.370268,11.847534,7445.788413,80.023148,1.844651,depressao
3,1.036585,3.054759,7.285360,1.364078,0.349553,0.747253,2.449669,0.818067,0.312195,0.761071,...,0.268973,0.051220,0.068108,0.362283,0.195190,0.331707,0.816800,2.310174,0.673331,depressao
4,1.327869,0.799606,9.443182,1.267385,0.699454,0.860492,5.022727,1.172748,0.060109,0.089283,...,0.000000,0.010929,0.010810,0.045455,0.060228,0.202186,0.368957,1.437500,0.507957,controle


# Codificando a variável categórica

In [5]:
label_encoder = LabelEncoder()
df['classe'] = label_encoder.fit_transform(df['classe']).astype('float64')

# Preparando os dados

In [6]:
def train_val_test(X, y, train_ratio, val_ratio, test_ratio):
    assert sum([train_ratio, test_ratio, val_ratio])==1.0, "wrong given ratio, all ratios have to sum to 1.0"
    assert X.shape[0]==len(y), "X and y shape mismatch"

    ind_train = int(round(X.shape[0]*train_ratio))
    ind_test = int(round(X.shape[0]*(train_ratio+test_ratio)))

    X_train = X[:ind_train]
    X_test = X[ind_train:ind_test]
    X_val = X[ind_test:]

    y_train = y[:ind_train]
    y_test = y[ind_train:ind_test]
    y_val = y[ind_test:]

    return X_train, y_train, X_val, y_val, X_test, y_test

In [7]:

def avaliacao(classificador, predicao, x, y, nome_classificador):
    '''
    Computa as mericas de avaliação dos clasficadores
    '''
        
    precisao = precision_score(y, predicao)
    revocacao = recall_score(y, predicao)
    f1score = f1_score(y, predicao)
    acuracia = accuracy_score(y, predicao)
    return [precisao, revocacao, f1score, acuracia]


    
def modeloML(model, x, x_train, y_train, x_val, y_val, nome_classificador, atributo, i):
    clear_output()
    
    print(f'{atributo}\nExperimento {i}')
    print(nome_classificador)
    
    model.fit(x_train, y_train)
    predicao = cross_val_predict(model, x_val, y_val, cv=cv)
    #predicao = GridSearchCV(model, {}, cv=cv, verbose=10, n_jobs=3).fit(x_train, y_train).predict(x_val)
    #predicao = cross_val_predict(GridSearchCV(model, {}, cv=cv, verbose=10).fit(x_train, y_train).best_estimator_, x_val, y_val, cv=cv)
    resultados = avaliacao(model, predicao, x_val, y_val, nome_classificador)
    df_resultados.loc[nome_classificador][f'DeChoudhury_+_{atributo}_precisao'] = round(resultados[0] * 100, 1)
    df_resultados.loc[nome_classificador][f'DeChoudhury_+_{atributo}_recall'] = round(resultados[1] * 100, 1)
    df_resultados.loc[nome_classificador][f'DeChoudhury_+_{atributo}_f1_score'] = round(resultados[2] * 100, 1)
    df_resultados.loc[nome_classificador][f'DeChoudhury_+_{atributo}_acuracia'] = round(resultados[3] * 100, 1)
    
    #plotarMatrizConfusao(y_val, predicao, nome_classificador)

In [8]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)
zscore = StandardScaler()

train_ratio = 0.70
validation_ratio = 0.15
test_ratio = 0.15

In [9]:
if not os.path.exists(f'df_resultados_{periodo}_novosAtributos_f1Score.xlsx'):
    df_choudhury = pd.read_excel(f'df_resultados_{periodo}_f1Score.xlsx', index_col=0)
    df_resultados = df_choudhury.loc[['Gradient Boosting', 'Perceptron Multicamadas']]
    atributos = ['caracteresOrientais', 'emojis', 'links','midia', 'curtidas']
    metricas = ['acuracia', 'precisao', 'recall', 'f1_score']
    colunas = [f'DeChoudhury_+_{i}_{j}' for i in atributos for j in metricas]
    df_resultados_novosAtributos = pd.DataFrame(columns=colunas, index=['Gradient Boosting', 'Perceptron Multicamadas'], data=np.nan)
    df_resultados = pd.concat([df_resultados, df_resultados_novosAtributos], axis=1, join='inner')
    colunas = [f'DeChoudhury_+_todosAtributos_{i}'for i in metricas]
    df_resultados_novosAtributos = pd.DataFrame(columns=colunas, index=['Gradient Boosting', 'Perceptron Multicamadas'], data=np.nan)
    df_resultados = pd.concat([df_resultados, df_resultados_novosAtributos], axis=1, join='inner')
else:
    df_resultados = pd.read_excel(f'df_resultados_{periodo}_novosAtributos_f1Score.xlsx', index_col=0)

df_resultados

,exp_basico_DeChoudhury_acuracia,exp_basico_DeChoudhury_precisao,exp_basico_DeChoudhury_recall,exp_basico_DeChoudhury_f1_score,DeChoudhury_+_caracteresOrientais_acuracia,DeChoudhury_+_caracteresOrientais_precisao,DeChoudhury_+_caracteresOrientais_recall,DeChoudhury_+_caracteresOrientais_f1_score,DeChoudhury_+_emojis_acuracia,DeChoudhury_+_emojis_precisao,...,DeChoudhury_+_midia_recall,DeChoudhury_+_midia_f1_score,DeChoudhury_+_curtidas_acuracia,DeChoudhury_+_curtidas_precisao,DeChoudhury_+_curtidas_recall,DeChoudhury_+_curtidas_f1_score,DeChoudhury_+_todosAtributos_acuracia,DeChoudhury_+_todosAtributos_precisao,DeChoudhury_+_todosAtributos_recall,DeChoudhury_+_todosAtributos_f1_score
Gradient Boosting,75.5,74.4,90.0,81.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Perceptron Multicamadas,74.5,74.5,87.4,80.4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def executar(x, y, atributo, i):
    
    x_train, y_train, x_val, y_val, x_test, y_test = train_val_test(x, y, train_ratio, validation_ratio, test_ratio)
    
    naiveBayes = BernoulliNB(alpha=1e-09)
    gradientBoosting = GradientBoostingClassifier(random_state=42, n_estimators=100, verbose=10)
    perceptronMulticamadas = MLPClassifier(random_state=42, hidden_layer_sizes=(10,30,10), max_iter=100, verbose=10)
    
    modeloML(perceptronMulticamadas, x, x_train, y_train, x_val, y_val, 'Perceptron Multicamadas', atributo, i)
    modeloML(gradientBoosting, x, x_train, y_train, x_val, y_val, 'Gradient Boosting', atributo, i)

In [11]:
choq = ['volumeTweets_media', 'volumeTweets_mediaMovelPonterada', 
        'indiceInsonia_variancia', 'indiceInsonia_mediaMovelPonterada',
        'pronome1Pessoa_media', 'pronome1Pessoa_variancia', 
        'pronome2Pessoa_mediaMovelPonterada', 'pronome2Pessoa_entropia',
        'pronome3Pessoa_media', 'pronome3Pessoa_mediaMovelPonterada',
        'valencia_mediaMovelPonterada', 'valencia_entropia', 
        'ativacao_mediaMovelPonterada', 'ativacao_entropia', 
        'termosDepressivos_variancia', 'termosDepressivos_mediaMovelPonterada',
        'grafoSocial_variancia', 'grafoSocial_mediaMovelPonterada',
        'medicamentosAntiDepressivo_media', 'medicamentosAntiDepressivo_mediaMovelPonterada']
caracteresOrientais = ['caracteresOrientais_variancia', 'caracteresOrientais_mediaMovelPonterada']
emojis = ['emojis_variancia', 'emojis_entropia']
links = ['links_mediaMovelPonterada', 'links_entropia']
midia = ['midia_variancia', 'midia_mediaMovelPonterada']
curtidas = ['curtidas_media', 'curtidas_mediaMovelPonterada']

In [12]:
exp1 = choq + caracteresOrientais
exp2 = choq + emojis
exp3 = choq + links
exp4 = choq + midia
exp5 = choq + curtidas
exp6 = choq + caracteresOrientais + emojis + links + midia + curtidas

exps = [exp1, exp2, exp3, exp4, exp5, exp6]
atributos = ['caracteresOrientais', 'emojis', 'links', 'midia', 'curtidas', 'todosAtributos']

lista = list(zip(exps, atributos))

for i, (exp, atributo) in enumerate(lista, 1):
    x = df[exp].copy()
    y = df['classe'].copy()
    x_new = zscore.fit_transform(x)
    executar(x_new, y, atributo, i)

df_resultados_round = df_resultados.copy()
df_resultados_round.to_excel(f'df_resultados_{periodo}_novosAtributos_f1Score.xlsx', encoding='utf-8')

todosAtributos
Experimento 6
Gradient Boosting
      Iter       Train Loss   Remaining Time 
         1           1.3386           43.31s
         2           1.2995           42.87s
         3           1.2666           42.90s
         4           1.2388           42.42s
         5           1.2145           41.89s
         6           1.1940           41.40s
         7           1.1754           40.92s
         8           1.1587           40.63s
         9           1.1453           39.99s
        10           1.1325           39.67s
        11           1.1213           39.20s
        12           1.1115           38.74s
        13           1.1023           38.28s
        14           1.0943           37.83s
        15           1.0869           37.46s
        16           1.0800           37.00s
        17           1.0745           36.63s
        18           1.0686           36.17s
        19           1.0634           35.71s
        20           1.0585           35.26s
       

        81           0.9299            1.46s
        82           0.9292            1.39s
        83           0.9282            1.31s
        84           0.9268            1.23s
        85           0.9252            1.16s
        86           0.9237            1.08s
        87           0.9228            1.00s
        88           0.9218            0.92s
        89           0.9209            0.85s
        90           0.9204            0.77s
        91           0.9190            0.69s
        92           0.9179            0.62s
        93           0.9171            0.54s
        94           0.9159            0.46s
        95           0.9146            0.39s
        96           0.9141            0.31s
        97           0.9134            0.23s
        98           0.9122            0.15s
        99           0.9112            0.08s
       100           0.9103            0.00s
      Iter       Train Loss   Remaining Time 
         1           1.3041            7.73s
         

        64           0.9477            2.77s
        65           0.9463            2.69s
        66           0.9456            2.62s
        67           0.9445            2.54s
        68           0.9433            2.46s
        69           0.9416            2.39s
        70           0.9396            2.31s
        71           0.9382            2.23s
        72           0.9372            2.16s
        73           0.9352            2.08s
        74           0.9332            2.00s
        75           0.9320            1.93s
        76           0.9308            1.85s
        77           0.9290            1.77s
        78           0.9280            1.70s
        79           0.9269            1.62s
        80           0.9258            1.54s
        81           0.9246            1.46s
        82           0.9233            1.39s
        83           0.9222            1.31s
        84           0.9213            1.23s
        85           0.9198            1.16s
        86

        47           0.9795            4.12s
        48           0.9781            4.05s
        49           0.9761            3.97s
        50           0.9743            3.89s
        51           0.9722            3.81s
        52           0.9711            3.74s
        53           0.9697            3.66s
        54           0.9678            3.58s
        55           0.9663            3.50s
        56           0.9651            3.43s
        57           0.9638            3.35s
        58           0.9627            3.27s
        59           0.9614            3.20s
        60           0.9598            3.12s
        61           0.9585            3.05s
        62           0.9569            2.97s
        63           0.9551            2.89s
        64           0.9541            2.81s
        65           0.9527            2.73s
        66           0.9510            2.66s
        67           0.9503            2.58s
        68           0.9488            2.50s
        69

        30           1.0239            5.32s
        31           1.0218            5.25s
        32           1.0193            5.18s
        33           1.0161            5.11s
        34           1.0139            5.03s
        35           1.0113            4.96s
        36           1.0089            4.89s
        37           1.0062            4.82s
        38           1.0040            4.74s
        39           1.0024            4.67s
        40           1.0000            4.59s
        41           0.9979            4.52s
        42           0.9959            4.44s
        43           0.9945            4.37s
        44           0.9930            4.28s
        45           0.9915            4.20s
        46           0.9898            4.13s
        47           0.9877            4.05s
        48           0.9864            3.98s
        49           0.9851            3.90s
        50           0.9833            3.83s
        51           0.9817            3.75s
        52

        13           1.0952            6.69s
        14           1.0881            6.62s
        15           1.0820            6.55s
        16           1.0758            6.48s
        17           1.0707            6.41s
        18           1.0653            6.33s
        19           1.0613            6.26s
        20           1.0564            6.12s
        21           1.0524            6.05s
        22           1.0491            5.98s
        23           1.0447            5.91s
        24           1.0410            5.84s
        25           1.0377            5.77s
        26           1.0339            5.69s
        27           1.0301            5.62s
        28           1.0268            5.54s
        29           1.0244            5.47s
        30           1.0215            5.36s
        31           1.0182            5.29s
        32           1.0155            5.21s
        33           1.0138            5.14s
        34           1.0115            5.07s
        35

        96           0.9218            0.31s
        97           0.9210            0.23s
        98           0.9198            0.16s
        99           0.9194            0.08s
       100           0.9186            0.00s


In [13]:
colunasFf1Score = [col for col in df_resultados.columns if 'f1_score' in col]
df_resultados_round[colunasFf1Score].style.set_precision(1)

,exp_basico_DeChoudhury_f1_score,DeChoudhury_+_caracteresOrientais_f1_score,DeChoudhury_+_emojis_f1_score,DeChoudhury_+_links_f1_score,DeChoudhury_+_midia_f1_score,DeChoudhury_+_curtidas_f1_score,DeChoudhury_+_todosAtributos_f1_score
Gradient Boosting,81.5,81.5,81.8,81.5,81.4,81.5,81.9
Perceptron Multicamadas,80.4,80.3,80.8,80.5,80.6,80.6,80.3
